In [3]:
pip  install -U farasapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import random 
import json
import pickle 
import numpy as np
#import tensorflow
import tensorflow 
from tensorflow import keras
print('tensorflow version', tensorflow.__version__)

tensorflow version 2.8.0


In [5]:
import nltk

In [6]:
from farasa.pos import FarasaPOSTagger
from farasa.ner import FarasaNamedEntityRecognizer
from farasa.diacratizer import FarasaDiacritizer
from farasa.segmenter import FarasaSegmenter
from farasa.stemmer import FarasaStemmer

In [ ]:
intents = json.loads(open('/content/intents.json', encoding="utf8").read())


In [7]:
stemmer = FarasaStemmer()

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


100%|██████████| 241M/241M [00:14<00:00, 16.6MiB/s]


In [14]:
from tensorflow.keras.models import load_model


intents = json.loads(open('intents.json').read())

words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
model = load_model('/content/SAUL_ChatbotModel.h5')

In [15]:
def clean_up_sentence(sentence):
    sentence_words = nltk.tokenize.wordpunct_tokenize(sentence)
    sentence_words = [stemmer.stem(word) for word in sentence_words]
    return sentence_words

In [16]:
#to convert sentence into bag of words (list full of 0's and 1's) to indicate if w is there or not
def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w: 
                bag[i] = 1
    return np.array(bag)

In [17]:
def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

In [18]:
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

In [22]:
print("GO! SAUL is running!")

while True: 
    message = input("")
    ints = predict_class(message)
    res = get_response(ints, intents)
    print(res)


GO! SAUL is running!
هاي 
هاي أساعدك ازاي؟
ايه الورق المطلوب فالكلية عشان اقدم ؟
 استمارة النجاح في الثانوية العامة أو مستخرج رسمي منها, شهادة الميلاد أو مستخرج رسمي منها , صورة البطاقة الشخصية أو جواز السفر, عدد 4 صور شخصية (46) , سداد رسوم التقديم واختبارات القبول(غير مستردة)  200 دولار , 1000 دولار من المصروفات الدراسية (تسترد بالكامل قبل   27/8/2020  ويسترد 50% فقط بعد هذا التاريخ وحتى الاسبوع الثالث من الدراسة ولا يسترد اى مصروفات بعد ذلك)
خطوط الباص في فرع العلمين ؟
كل الباصات بتتحرك الساعة 3 من فرع العلمين وموجود 4 خطوط مختلفه ( خط الحريه , خط الساحل الشمالى , خط الكورنيش , خط سموحه)


KeyboardInterrupt: ignored